In [1]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score, f1_score
import time
import numpy as np

In [2]:
# Set paths for training and test data
train_dir = r'D:\Onedrive\Desktop\L6\fruits-360-original-size\Training'
test_dir = r'D:\Onedrive\Desktop\L6\fruits-360-original-size\Test'

In [3]:
import os  # Ensure os is imported

IMG_SIZE = (224, 224)
BATCH_SIZE = 64
NUM_CLASSES = len(next(os.walk(train_dir))[1])  # Number of classes


In [4]:
# Load DenseNet201 as base model
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the DenseNet201 model

In [5]:
# Add a custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
output_layer = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 4821 images belonging to 15 classes.
Found 2408 images belonging to 15 classes.


In [8]:
# Train the model and measure training time
start_time = time.time()
model.fit(train_data, epochs=5, validation_data=test_data)
train_time = time.time() - start_time

c:\Users\LEGION\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 435s 6s/step - accuracy: 0.4928 - loss: 1.8242 - val_accuracy: 0.9817 - val_loss: 0.4464
Epoch 2/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 401s 5s/step - accuracy: 0.9824 - loss: 0.3675 - val_accuracy: 0.9996 - val_loss: 0.2024
Epoch 3/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 402s 5s/step - accuracy: 0.9988 - loss: 0.1756 - val_accuracy: 1.0000 - val_loss: 0.1187
Epoch 4/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 400s 5s/step - accuracy: 1.0000 - loss: 0.1095 - val_accuracy: 1.0000 - val_loss: 0.0803
Epoch 5/5
76/76 ━━━━━━━━━━━━━━━━━━━━ 393s 5s/step - accuracy: 1.0000 - loss: 0.0739 - val_accuracy: 1.0000 - val_loss: 0.0582


In [9]:
# Evaluate the model and calculate evaluation metrics
print("Evaluating DenseNet201 model...")
start_eval_time = time.time()
loss, accuracy = model.evaluate(test_data)
eval_time = time.time() - start_eval_time

Evaluating DenseNet201 model...
38/38 ━━━━━━━━━━━━━━━━━━━━ 136s 4s/step - accuracy: 1.0000 - loss: 0.0574


In [10]:
# Calculate F1-score
test_data.reset()  # Reset the generator for prediction
preds = model.predict(test_data)
y_true = test_data.classes
y_pred = np.argmax(preds, axis=1)
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print(f"Training Time: {train_time:.2f} seconds")
print(f"Evaluation Time: {eval_time:.2f} seconds")

38/38 ━━━━━━━━━━━━━━━━━━━━ 148s 4s/step
Test Accuracy: 100.00%
F1 Score: 0.07
Training Time: 2033.08 seconds
Evaluation Time: 136.51 seconds


In [14]:
# Save the model
model.save(r'D:\Onedrive\Desktop\L6\APPLEVARIETIES\models\densenet201_classifier.keras')
print("Model saved at 'D:\\Onedrive\\Desktop\\L6\\APPLEVARIETIES\\models\\densenet201_classifier.keras'")

Model saved at 'D:\Onedrive\Desktop\L6\APPLEVARIETIES\models\densenet201_classifier.keras'
